In [1]:
import os
import torch
import torchvision
from torchsummary import summary
from matplotlib import pyplot as plt
from torch.utils.data import DataLoader as dataloader
from tensorboardX import SummaryWriter
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn
import numpy as np
import time

# 设定参数

In [2]:
# hyper parameters
path_model = "./checkpoint/"
batch_size = 512
epochs = 15
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
# sampler = torch.utils.data.SubsetRandomSampler(indices=list(range(2000)))

# download mnist dataset
dataset_train = torchvision.datasets.MNIST(root='./data/',train=True,download=True,transform=transform)
dataset_test = torchvision.datasets.MNIST(root='./data/',train=False,download=True,transform=transform)

class_names = dataset_train.classes  # 获取数据集的分类信息 返回一个字典
# load dataset
data_train = dataloader(dataset=dataset_train,batch_size=batch_size,shuffle=False)
data_test = dataloader(dataset=dataset_test,batch_size=batch_size,shuffle=True)


# 定义训练过程

In [58]:
def test():
    net.eval()  # 切换到测试模式
    test_correct_num = 0
    with torch.no_grad():   # 不更新参数

        for batch_idx,(data,target) in enumerate(data_test):
            # data = data.to(device)
            # target = target.to(device)
            output = net(data) # 正向传播得到预测值
            _, pred = torch.max(output.data, 1)
            test_correct_num += torch.sum(pred==target).item()
            print("Test Epoch:{} [{}/{} ({:.0f}%)]\t acc:{:.2f}".format(epoch,batch_idx*batch_size,len(data_test.dataset),
                                                 100. * batch_size*batch_idx/len(data_test.dataset),test_correct_num/len(data_test.dataset)))
def train():

    for batch_idx, (data, target) in enumerate(data_train):
        # 清除grad累积值
        optimizer.zero_grad()
        # 读取dataloader中的数据，前半部分是tensor变量，后半部分是真实label
        data = data.to(device)
        target = target.to(device)
        # forward之后得到预测值
        output = net(data)
        # 计算loss
        loss = cost_fun(output, target)
        # backward
        loss.backward()
        # 收集一组新的梯度，并使用optimizer.step()将其传播回每个网络参数
        optimizer.step()
        # 给出loss和acc
        train_loss.append(loss)
        _, pred = torch.max(output.data, 1)
        correct_num = torch.sum(pred == target).item()
        train_acc.append(correct_num / batch_size)
        print("Train Epoch:{}[{}/{} ({:.0f}%)]\t Loss:{:.6f} acc:{:.2f}".format(epoch, batch_idx * batch_size,
               len(data_train.dataset),100. * batch_size * batch_idx / len(data_train.dataset), loss.item(),correct_num / batch_size))

def save_state():
    print('===> Saving weights...')
    state = {
        'state': net.state_dict(),
        'epoch': epoch  # 将epoch一并保存
    }
    if not os.path.isdir('checkpoint'):
        os.mkdir('./checkpoint')
    torch.save(state, path_model + 'Epoch:' + str(epoch) + ' Loss:' + str(train_loss[-1].item()) + '.pth')

def predict():
    state_path = './checkpoint/model_14.pth' #  ***为指定加载的权重文件名称
    print('===> Loading weights : ' + state_path)
    weight_dict = torch.load(state_path)  # 加载最后训练出的权重
    net.load_state_dict(weight_dict['state'])
    # 从测试集中选取一个batch做预测
    # pred_test = enumerate(data_test)
    # batch_idx, (pred_data, pred_gt) = next(pred_test)
    # output = net(pred_data)
    # print("data: ", output.data)
    # maxdata, pred = torch.max(output.data, 1) # 得到预测值,返回每一行的最大值，且返回索引
    # print("maxdata: ", maxdata)
    # print("ground truth: ",pred_gt)
    # print("predict value: ",pred)
    # 获取预测结果
    classes = [
        "0",
        "1",
        "2",
        "3",
        "4",
        "5",
        "6",
        "7",
        "8",
        "9",
    ]


    # 把tensor转成Image， 方便可视化
    show = ToPILImage()

    device = "cuda" if torch.cuda.is_available() else 'cpu'
    net.eval()
    for i in np.random.randint(0,20,size=10):
        x, y = dataset_test[i][0], dataset_test[i][1]
        # tensor格式数据可视化
        show(x).show()
        # 扩展张量维度为4维
        x = Variable(torch.unsqueeze(x, dim=0).float(), requires_grad=False).to(device)
        with torch.no_grad():
            pred = net(x)
            # 得到预测类别中最高的那一类，再把最高的这一类对应classes中的哪一类标签
            predicted, actual = classes[torch.argmax(pred[0])], classes[y]
            # 最终输出的预测值与真实值
            print(f'predicted: "{predicted}", actual:"{actual}"')


# 构建网络

In [49]:
# 构建网络
class LeNet(nn.Module): 					# 继承于nn.Module这个父类
    def __init__(self):						# 初始化网络结构
        super(LeNet, self).__init__()    	# 多继承需用到super函数
        self.block_1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1, padding=2),  # 输出为6*28*28
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # 输出为6*14*14
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1),  # 输出为16*10*10
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # 输出为16*5*5
        )
        self.block_2 = nn.Sequential(
            nn.Linear(16*5*5, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, 10),
        )

    def forward(self, x):  # 正向传播过程
        x = self.block_1(x)
        x = x.view(-1,16*5*5)
        x = self.block_2(x)
        return x


In [5]:
def variaes_show():
    net = LeNet()
    data_input = Variable(torch.randn(16,1,28,28))
    print(data_input.size())
    net(data_input)
    print(summary(net,(1,28,28)))

# 主函数

In [6]:
if __name__ == '__main__':
    writer = SummaryWriter('logs')
    # device = torch.device('cuda:0')

    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu')
    
    net = LeNet()  # 实例化网络
    # data_input = Variable(torch.randn(16,1,28,28))
    # print(net(data_input))
    net.to(device) # 将参数送入GPU中

    def init_weights(m):
        if type(m) == nn.Linear or type(m) == nn.Conv2d:
            torch.nn.init.xavier_uniform_(m.weight)

    net.apply(init_weights)

    cost_fun = nn.CrossEntropyLoss()
    # optim
    optimizer = torch.optim.SGD(net.parameters(), lr=1e-3, momentum=0.95, weight_decay=1e-3)

    # train
    for epoch in range(epochs):

        # train
        train_loss = []
        train_acc = []
        # train
        train()
        writer.add_scalar('Train/Loss', train_loss[-2].item(), epoch)
        writer.add_scalar('Train/Acc', train_acc[-2], epoch)

        # ----------------------------------------- #
        # save_state
        # ----------------------------------------- #
        print('===> Saving models...')
        state = {
            'state': net.state_dict(),
            'epoch': epoch  # 将epoch一并保存
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('./checkpoint')
        torch.save(state, path_model + 'Epoch-' + str(epoch) + '-Loss-'+ str(train_loss[-1].item()) + '.pth')

        # ----------------------------------------- #
        # test
        # ----------------------------------------- #
        test()
    writer.close()

    # ----------------------------------------- #
    # 加载指定的weights进行预测
    # ----------------------------------------- #
    # predict()

Train Epoch:0[0/60000 (0%)]	 Loss:2.315984 acc:0.08
Train Epoch:0[512/60000 (1%)]	 Loss:2.316641 acc:0.08
Train Epoch:0[1024/60000 (2%)]	 Loss:2.308957 acc:0.10
Train Epoch:0[1536/60000 (3%)]	 Loss:2.318130 acc:0.08
Train Epoch:0[2048/60000 (3%)]	 Loss:2.308060 acc:0.09
Train Epoch:0[2560/60000 (4%)]	 Loss:2.309898 acc:0.10
Train Epoch:0[3072/60000 (5%)]	 Loss:2.311486 acc:0.08
Train Epoch:0[3584/60000 (6%)]	 Loss:2.315276 acc:0.09
Train Epoch:0[4096/60000 (7%)]	 Loss:2.307842 acc:0.08
Train Epoch:0[4608/60000 (8%)]	 Loss:2.304930 acc:0.09
Train Epoch:0[5120/60000 (9%)]	 Loss:2.308060 acc:0.09
Train Epoch:0[5632/60000 (9%)]	 Loss:2.313003 acc:0.10
Train Epoch:0[6144/60000 (10%)]	 Loss:2.300329 acc:0.09
Train Epoch:0[6656/60000 (11%)]	 Loss:2.302673 acc:0.10
Train Epoch:0[7168/60000 (12%)]	 Loss:2.307391 acc:0.10
Train Epoch:0[7680/60000 (13%)]	 Loss:2.304505 acc:0.07
Train Epoch:0[8192/60000 (14%)]	 Loss:2.305594 acc:0.08
Train Epoch:0[8704/60000 (15%)]	 Loss:2.296808 acc:0.10
Train Ep

# 测试

In [57]:
from torchvision.transforms import ToPILImage

net = LeNet()
predict()

===> Loading weights : ./checkpoint/model_14.pth
data:  tensor([[ 9.1946, -5.7479,  2.5912,  ...,  1.2633,  1.7489,  2.1585],
        [-0.1924, -5.7894, -0.4259,  ...,  2.5880, 10.7342,  7.3210],
        [-1.5003, -3.4130, -0.1546,  ..., -2.7477,  4.5763,  2.6126],
        ...,
        [ 2.2570, -4.9382, -1.7769,  ..., -2.2978, 13.0141,  3.4117],
        [-3.0545, -3.3421,  0.6359,  ...,  2.4441,  0.6022,  7.6658],
        [-0.0940,  3.8249,  4.3698,  ...,  7.4604,  2.0343,  2.2426]])
maxdata:  tensor([ 9.1946, 10.7342, 11.5637,  8.6294, 14.4509, 11.7617, 10.9212,  5.5727,
        10.8699, 10.9189,  9.1539,  8.4965, 10.2559,  8.9639,  6.8700,  7.6728,
        10.3321, 12.8395,  8.6964, 11.7861, 14.7114, 14.2235, 10.6771, 11.5794,
        14.3558, 15.1084, 17.8112, 10.7303, 16.8310, 18.8622, 13.5385,  8.7315,
         6.9688, 13.2827, 14.8343, 19.2038, 13.5622, 12.5825, 13.1289, 17.8454,
        16.7839, 13.4893, 12.1894, 10.0549,  9.2326,  9.8891,  9.9304, 10.9707,
        14.9364, 17.

# 读取权重

In [7]:
import torch
import torchvision
import numpy as np
import os
from matplotlib import pyplot as plt
import pandas as pd

In [53]:
def load_weight(model_path):
        
        print('===> Loading weights : ' + model_path)
        weight_dict = torch.load(model_path)  # 加载最后训练出的权重

        print('type: ' + str(type(weight_dict)))
        print('len: ' + str(len(weight_dict)))

        for k in weight_dict.keys():
                print('key: '+ k)

        # print(weight_dict['state'])
        # print(weight_dict['epoch'])


        for key,value in weight_dict['state'].items():
                value_np = value.numpy()
                if not os.path.isdir('csv'):
                        os.mkdir('./csv')
                # np.savetxt("./csv/%s.csv" %(key), value_np,  delimiter=",")
                # pd.DataFrame(value_np).to_csv("./csv/%s.csv" %(key))
                # print(key, value.size())
                print('shape: '+str(value_np.shape))
                if (value_np.ndim == 4):
                        (n_dim, _, _, _) = value_np.shape
                        value_2d = value_np.reshape(n_dim,-1)
                        np.savetxt("./csv/%s.csv" %(key), value_2d,  delimiter=",")
                        print(value_2d.shape)
                elif (value_np.ndim == 3):
                        ndim, _, _ = value_np.shape
                        value_2d = value_np.reshape(n_dim,-1)
                        np.savetxt("./csv/%s.csv" %(key), value_2d,  delimiter=",")
                        print(value_2d.shape)
                # elif (value_np.ndim == 2):
                #         ndim, _ = value_np.shape
                #         value_2d = value_np.reshape(n_dim,-1)
                #         print(value_2d.shape) 
                else :  
                        value_2d = value_np
                        np.savetxt("./csv/%s.csv" %(key), value_2d,  delimiter=",")
                        print(value_2d.shape)             



        print(type(weight_dict['state']))
        

state_path = './checkpoint/model_14.pth'
load_weight(state_path)

===> Loading weights : ./checkpoint/model_14.pth
type: <class 'dict'>
len: 2
key: state
key: epoch
shape: (6, 1, 5, 5)
(6, 25)
shape: (6,)
(6,)
shape: (16, 6, 5, 5)
(16, 150)
shape: (16,)
(16,)
shape: (120, 400)
(120, 400)
shape: (120,)
(120,)
shape: (84, 120)
(84, 120)
shape: (84,)
(84,)
shape: (10, 84)
(10, 84)
shape: (10,)
(10,)
<class 'collections.OrderedDict'>
